<a href="https://colab.research.google.com/github/davidmouquetGit/DemoModelisation/blob/master/Conso_logement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
from datetime import datetime
drive.mount('/content/drive')
myDir = '/content/drive/My Drive/Consommation logement'

Mounted at /content/drive


In [ ]:
import os
dir_data_elec_jour = os.path.join(myDir,'données élec/jour')
dir_data_elec_hour = os.path.join(myDir,'données élec/horaires')
dir_data_gaz       = os.path.join(myDir,'données gaz')

# Météo

In [ ]:
data_meteo = pd.read_csv(os.path.join(myDir,'météo/Text.csv'),sep=";",decimal=",")
data_meteo['Date'] = data_meteo['Date'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y %H:%M:%S"))
data_meteo.set_index('Date',inplace=True)

# Electricite

## Lecture des données

### données horaires

In [ ]:
import os

def lister_fichiers_par_extension(repertoire, extension):
  list_files = list()
  for racine, repertoires, fichiers in os.walk(repertoire):
      for fichier in fichiers:
          if fichier.endswith(extension):
              list_files.append(os.path.join(racine, fichier))
  return list_files

liste_fichier_elec_horaire = lister_fichiers_par_extension(dir_data_elec_hour, '.xlsx')
list_data = list()
for fi in liste_fichier_elec_horaire:
  data = pd.read_excel(fi,usecols="C:E",skiprows=15,sheet_name="Consommation Horaire")
  list_data.append(data)

data_elec_h = pd.concat(list_data)
data_elec_h.drop_duplicates(inplace=True)
data_elec_h['Valeur (en kW)'] = data_elec_h['Valeur (en kW)'].apply(lambda c: float(c.replace(',','.')))
data_elec_h['Début'] = data_elec_h['Début'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y %H:%M:%S"))

data_elec_h.index = data_elec_h['Début']
data_elec_h.sort_index(inplace=True)

data_elec_h = data_elec_h['Valeur (en kW)'].to_frame()

### Données jour

In [ ]:
import pandas as pd


data_elec_j = pd.read_excel(os.path.join(dir_data_elec_jour,"enedis.xlsx"))

data_elec_j['Horodate'] = data_elec_j['Horodate'].apply(lambda x: datetime.strptime(x[0:-6], "%Y-%m-%dT%H:%M:%S"))
data_elec_j.columns = ['Horodate','index_hc','index_hp','nbre_pers']
data_elec_j['conso_hc_kwh'] = (data_elec_j['index_hc']-data_elec_j['index_hc'].shift(1))/1000.0
data_elec_j['conso_hp_kwh'] = (data_elec_j['index_hp']-data_elec_j['index_hp'].shift(1))/1000.0
data_elec_j['conso_kwh']    = data_elec_j['conso_hc_kwh'] + data_elec_j['conso_hp_kwh']
data_elec_j.set_index('Horodate',inplace=True)

data_elec_j = data_elec_j[['conso_hc_kwh',	'conso_hp_kwh',	'conso_kwh','nbre_pers']].resample('D').mean()
data_elec_j.index.name = 'Date'

Text_jour = data_meteo['Text'].resample('D').mean()

data_elec_j = data_elec_j.join(Text_jour)
data_elec_j.dropna(inplace=True)

In [ ]:
conso_hp = data_elec_j['conso_hp_kwh'].sum()
conso_hc = data_elec_j['conso_hc_kwh'].sum()


In [ ]:
conso_hp

np.float64(21118.340000000004)

In [ ]:
conso_hp/(conso_hp+conso_hc)

np.float64(0.5130133439623543)

## Modèle de consommation électrique journalier

In [ ]:
data_ref = data[(data.index>='2023-01-01') & (data.index<='2023-12-31')]

In [ ]:
from sklearn.linear_model import LinearRegression

model_elec = LinearRegression()
model_elec.fit(X=data_ref['Text'].values.reshape(-1, 1),y=data_ref['conso_kwh'].values.reshape(-1, 1))
print(model_elec.score(X=data_ref['Text'].values.reshape(-1, 1),y=data_ref['conso_kwh'].values.reshape(-1, 1)))

LinearRegression()

In [ ]:
print(model_elec.score(X=data_ref['Text'].values.reshape(-1, 1),y=data_ref['conso_kwh'].values.reshape(-1, 1)))

0.4492589015596735


In [ ]:
data['model_elec'] = model_elec.predict(data['Text'].values.reshape(-1, 1))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['conso_kwh'],
                    mode='lines',
                    name='conso elec'))
fig.add_trace(go.Scatter(x=data.index, y=data['model_elec'],
                    mode='lines',
                    name='modele conso elec'))

fig.show()

In [ ]:
data.head()

,conso_hc_kwh,conso_hp_kwh,conso_kwh,nbre_pers,Text,model_elec
Date,,,,,,
2021-12-22,28.548,38.366,66.914,4.0,-1.945833,54.587890
2021-12-23,32.545,40.715,73.260,4.0,3.654167,46.435296
2021-12-24,25.627,48.730,74.357,4.0,8.566667,39.283578
2021-12-25,20.132,46.965,67.097,4.0,9.129167,38.464679
2021-12-26,12.310,38.928,51.238,4.0,9.166667,38.410086


In [ ]:
data_month = data[(data.index>='2022-01-01') & (data.index<='2024-11-30')]

In [ ]:
data_month = data_month.resample("1M").sum()

<ipython-input-47-870a90468a70>:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
data_month.index[0].year

2022

In [ ]:
data_month['Date'] = data_month.index

In [ ]:
data_month['Mois'] = data_month['Date'].apply(lambda d: d.strftime("%B"))
data_month['Période'] = data_month['Date'].apply(lambda d: d.strftime("%B") + " " + str(d.year))


In [ ]:
import plotly.express as px

fig = px.bar(data_month, x="Mois", y="conso_kwh",
             color='Période', barmode='group',
             height=400)
fig.show()

## Analyse des données horaires


In [ ]:
import plotly.graph_objects as go

# Create traces

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_elec_h.index, y=data_elec_h['Valeur (en kW)'],
                    mode='lines',
                    name='conso elec'))

fig.show()

In [ ]:
data_elec_h = data_elec_h.resample("h").mean()
data_elec_h['Date'] = data_elec_h.index
data_elec_h['Heure'] = data_elec_h['Date'].apply(lambda d: d.strftime("%H"))
data_elec_h['Jour'] = data_elec_h['Date'].apply(lambda d: d.strftime("%D"))
data_elec_hp = data_elec_h.pivot(index="Jour", columns="Heure", values="Valeur (en kW)")

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=data_elec_hp.values,
        x=data_elec_hp.columns,
        y=data_elec_hp.index.to_list(),
        colorscale='Viridis'
))


fig.show()

# Gaz

In [ ]:
import pandas as pd


data_gaz = pd.read_excel(os.path.join(dir_data_gaz,"Donnees_facturantes_02269609199626_2020-01-08_2024-12-18_677e730465765.xlsx"),skiprows=7,usecols='B:I')
data_gaz['Date de relevé'] = data_gaz['Date de relevé'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y"))
data_gaz['Date fin'] = data_gaz['Date de relevé'].shift(-1)
data_gaz = data_gaz[data_gaz['Date de relevé']>='2022-08-03']

In [ ]:
def text_moy(row):
  debut = row['Date de relevé']
  fin   = row['Date fin']
  text_periode = data_meteo[(data_meteo.index>=debut) & (data_meteo.index<=fin)]
  return text_periode['Text'].mean()

data_gaz['Text'] = data_gaz.apply(text_moy,axis=1)


<ipython-input-44-100d87afb7b2>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import plotly.express as px
fig = px.scatter(data_gaz, x="Text", y="Energie consommée (kWh)")
fig.show()

In [ ]:
data_ref = data_gaz[data_gaz['Text']<15]

In [ ]:
from sklearn.linear_model import LinearRegression

model_elec = LinearRegression()
model_elec.fit(X=data_ref['Text'].values.reshape(-1, 1),y=data_ref['Energie consommée (kWh)'].values.reshape(-1, 1))
print(model_elec.score(X=data_ref['Text'].values.reshape(-1, 1),y=data_ref['Energie consommée (kWh)'].values.reshape(-1, 1)))

0.8372334904708562


In [ ]:
model_elec.coef_[0][0]*0.07

-11.413367650460115

In [ ]:
data_gaz

,Date de relevé,Index de début de période (m3),Index de fin de période (m3),Volume consommé (m3),Coefficient de conversion,Energie consommée (kWh),Nature du relevé,Qualification du relevé,Date fin
8,2022-08-03,0,0,0,11.14,0,Publiée,Mesuré,2022-08-18
9,2022-08-18,0,3,3,11.16,33,Publiée,Mesuré,2022-09-01
10,2022-09-01,3,5,2,11.16,22,Publiée,Mesuré,2022-09-18
11,2022-09-18,5,27,22,11.19,246,Publiée,Mesuré,2022-10-01
12,2022-10-01,27,47,20,11.19,224,Publiée,Mesuré,2022-10-18
13,2022-10-18,47,50,3,11.17,34,Publiée,Mesuré,2022-11-01
14,2022-11-01,50,117,67,11.17,748,Publiée,Mesuré,2022-11-18
15,2022-11-18,117,230,113,11.15,1260,Publiée,Mesuré,2022-12-01
16,2022-12-01,230,458,228,11.15,2542,Publiée,Mesuré,2022-12-18
17,2022-12-18,458,595,137,11.16,1529,Publiée,Mesuré,2023-01-01
